In [1]:
import numpy as np
import scipy.sparse as sps
from ntf_cython.nmf import bpp as bpp

%matplotlib inline

## Synthetic Random Matrix

In [2]:
m = 10000
n = 1000
k = 100
A0 = np.random.randn(m, k)
A1 = np.random.randn(k, n)
A = A0.dot(A1)

In [3]:
def compute_norm(A, p=None, axis=None):
    
    if sps.issparse(A):
        return sps.linalg.norm(A, ord=p, axis=axis)
    else:
        return np.linalg.norm(A, ord=p, axis=axis)

In [4]:
def xray(A, r):
    
    cols= []
    R = A
    
    while len(cols) < r:
        
        #Find an extreme ray by looping until a column has been chosen which was not previously selected
        
        while True:
            p = np.random.random((1, A.shape[0]))
            scores = compute_norm(np.dot(R.T,A), axis = 0)
            scores = scores/(np.dot(p, A))
            scores[0, cols] = -1
            best_col = np.argmax(scores)
            
            if best_col in cols:
                continue
            else:
                cols.append(best_col)
                H = bpp(A[:, cols], A)
                R = A - np.dot(A[:, cols], H)
                break
    return cols

In [5]:
def SPA(A, r):
    
    """
    Based on this paper 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3927512/pdf/sensors-08-01321.pdf'
    """
    colnorms = compute_norm(A, p=1, axis=0)
    A = A/colnorms
    cols = []
    m = A.shape[0]
    for _ in range(r):
        col_norms = np.linalg.norm(A, axis=0)
        col_norms[cols] = -1
        col_idx = np.argmax(col_norms)
        cols.append(col_idx)
        col = np.reshape(A[:, col_idx], (m , 1))
        A = np.dot((np.eye(m) - np.dot(col, col.T) / col_norms[col_idx]), A)
    
    return cols
    

In [6]:
cols1 = SPA(A, 20)

In [17]:
cols1

[413,
 853,
 311,
 474,
 123,
 280,
 750,
 680,
 636,
 851,
 49,
 976,
 880,
 252,
 384,
 575,
 902,
 747,
 587,
 110]

In [18]:
cols2 = xray(A, 20)

In [19]:
cols2

[353,
 408,
 591,
 724,
 315,
 226,
 450,
 390,
 336,
 178,
 224,
 145,
 910,
 594,
 928,
 922,
 787,
 93,
 658,
 814]